# Chicken and egg in molecular metadynamics
Create protein folding landmarks from scratch

[Contents](index.ipynb)

Prev: [Generate random lanmarks](gen-landmarks.ipynb)
Next: [Train the ANN](anncolvar.ipynb)

## 2. Compute isomap projection of the landmarks


In [ ]:
# the same housekeeping
import os

try:
    workdir
    raise Exception("This cell should be run only once (workdir = %s)" % workdir)
except NameError:
    pass
   
basedir=os.getcwd()
workdir=basedir + '/work'

In [ ]:
# XXX: MUST be the same as in gen-landmarks

# number of steps to twist a dihedral
nsteps = 12
# number of iterations; alltogether niter * nsteps random conformers are generated
niter = 200



# number of nearest neighbours to consider (aka _k_)
neighs = 5
# targed no. of dimensions 
dims = 2

In [ ]:
import os
import mdtraj as md
import numpy as np
from scipy.sparse import coo_matrix

os.chdir(workdir)

nconf = niter * nsteps

# superpose all landmarks on the first one
tr = md.load([ "conf%d-min.gro" % i for i in range(1,nconf+1)])
idx = tr[0].top.select("name CA")
tr.superpose(tr[0],atom_indices=idx)

# consider heavy atoms only 
idx = tr[0].top.select("element != H")

# compute all-to-all RMSD and select _k_ closest neighbours
row=[]
col=[]
dat=[]

for i in range(nconf):
    d = md.rmsd(tr,tr,frame=i,atom_indices=idx)
    d[range(i+1)] = np.inf
    for _ in range(neighs):
        j = np.argmin(d)
        if d[j] < np.inf:
            row.append(i)
            col.append(j)
            dat.append(d[j])
            row.append(j)
            col.append(i)
            dat.append(d[j])
            d[j] = np.inf

# store results in sparse matrix
dist = coo_matrix((dat,(row,col)),shape=(nconf,nconf)) 

In [ ]:
# isomap itself: compute shortest paths in the k-neighbours graph, 
# and multi-dimensional scaling on the resulting all-to-all distances

from sklearn.utils.graph_shortest_path import graph_shortest_path
from sklearn.manifold import MDS

sp = graph_shortest_path(dist,directed=False)
mds = MDS(n_components=dims,dissimilarity='precomputed',n_jobs=-1)
emb = mds.fit_transform(sp)

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

plt.scatter(*emb.transpose(),marker='.')
plt.show()

In [ ]:
np.savetxt('colvar.txt',emb)